# Walkthrough of Linear Algebra Functionality

With this section we will explore how to use various linear algebra operations with an emphasis on the dense and iterative base cases, rather than the dispatch rules. For high level operations like `solve`, `sqrt`, `logdet`, `eigs`, `exp`, `trace` the specific functions we call are listed in  [high-level API](https://cola.readthedocs.io/en/latest/package/cola.linalg.html), and low level iterative functions that help implement these operations such as `cg`, `gmres`, `lanczos`, `approx_diag`, `stochastic_lanczos_quadrature` are listed in [algorithms](https://cola.readthedocs.io/en/latest/package/cola.algorithms.html). These low level algorithms can also be called explicitly, but they will not be able to leverage our dispatch rules, and they sometimes require more involvement and knowledge from the user.

First let's set up some Linear operators to test these operations on. As we want to focus on the base cases, we will use a simple low rank + diagonal linear operator. We will use $A=UU^T+D$ as a prototype for a PSD matrix, $B=UV^T+D$ as a prototype for a generic square matrix, and $C=V^TD$ as a prototype for a generic rectangular matrix.

In [1]:
import torch
import numpy as np
import cola

N=2000
U = cola.lazify(torch.randn(N,5))
V = cola.lazify(torch.randn(N,5))
D = cola.diag(torch.linspace(1,100,N))

A = U@U.T + D # a PSD matrix
B = U@V.T + D # a generic square matrix
C = V.T@D # a generic rectangular matrix

x = torch.ones(N) # test vector x

Sometimes CoLA is able to infer additional properties of a Linear Operator such as `PSD`, `SelfAdjoint`, or `Unitary` but not always, so it's best to annotate these properties explicitly.

In order to let CoLA know that `A` is PSD, we will annotate it with the PSD annotation:

In [2]:
print("Properties before annotating:", A.annotations)
A = cola.PSD(A)
print("Properties after annotating:", A.annotations)

Properties before annotating: set()
Properties after annotating: {PSD}


Let's plot the spectrum of $A$ to get a sense for the kind of object that we are dealing with.

Text(0.5, 0, 'index')

In [3]:
import matplotlib.pyplot as plt
plt.plot(torch.linalg.eigh(A.to_dense())[0])
plt.yscale('log')
plt.ylabel("eigenvalues")
plt.xlabel("index")

## Inverses / Linear Solves

For solving a linear system $Ax=b$, we may consider two cases where $A$ is a Positive-SemiDefinite symmetric matrix (PSD), and when it is not.

`cola.linalg.inverse(A)` represents the linear operator $A^{-1}$ that, when applied to a vector $b$, it solves the linear system $A x = b$ and then outputs $x$.
It does not, however, compute the inverse densely $A^{-1}$ and then multiply it by the vector $b$, and is exactly equivalent to calling `cola.linalg.solve` as shown below.


### PSD

In [4]:
# these two are exactly equivalent in CoLA
y = cola.linalg.solve(A,x)
y = cola.linalg.inv(A)@x

However with `cola.linalg.inv` we can examine the properties of the solve like the number of iterations it as taken to converge. Instead using the default `method=auto`, let's explicitly pick `method=dense` to use a dense $O(n^3)$ method or `method=iterative` to use an iterative $O(\tau \sqrt{\kappa}\log 1/\epsilon)$ where $\tau$ is the time for an MVM with $A$, $\kappa$ is the condition number of $A$ and $\epsilon$ is the desired error tolerance.

In [5]:
for method in ['dense','iterative']:
    Ainv = cola.linalg.inv(A, method=method)
    y = Ainv@x
    print(f"With {method}: Ainv of type {type(Ainv)}")
    if method == 'iterative':
        print(f"Computed inverse in {Ainv.info['iterations']} iters with error {Ainv.info['errors'][-1]:.1e}")

With dense: Ainv of type <class 'cola.ops.operators.Product[cola.linalg.inv.TriangularInv[cola.ops.operators.Triangular], cola.linalg.inv.TriangularInv[cola.ops.operators.Triangular]]'>
With iterative: Ainv of type <class 'cola.linalg.inv.CGInv'>
Computed inverse in 119 iters with error 8.7e-07


In the dense case the `inverse` LinearOperator is computed via Cholesky decomposition
$A = LL^T$, and then using $A^{-1} = L^{-T}L^{-1}$ where $L^{-1}$ simply means a Linear Operator that performs triangular solves with $L$. We can see this reflected in the type of `Ainv`.

Meanwhile `iterative` uses Conjugate Gradients to perform multiplies with $A^{-1}$. With the iterative methods one often may want to specify an error tolerance or a maximum number of iterations to limit the computation.

In [6]:
Ainv = cola.linalg.inv(A, method='iterative', tol=1e-3, max_iters=100)
y = Ainv@x
print(f"Computed inverse in {Ainv.info['iterations']} iters with error {Ainv.info['errors'][-1]:.1e}")

Computed inverse in 60 iters with error 8.4e-04


With this higher error tolerance, CG can converge in many fewer iterations, and in general one should choose error tolerances as high as are acceptable.

### Non PSD

Let's call the same functions but with the non PSD operator $B$.

In [7]:
for method in ['dense','iterative']:
    Binv = cola.linalg.inv(B, method=method, tol=1e-3, max_iters=100)
    y = Binv@x
    print(f"With {method}: Ainv of type {type(Binv)}")
    if method == 'iterative':
        print(f"Computed inverse in {Binv.info['iterations']} iters with error {Binv.info['errors'][-1]:.1e}")

print("Actual residual error:", torch.linalg.norm(B@y-x)/torch.linalg.norm(x))

With dense: Ainv of type <class 'cola.ops.operators.Product[cola.linalg.inv.TriangularInv[cola.ops.operators.Triangular], cola.linalg.inv.TriangularInv[cola.ops.operators.Triangular], cola.ops.operators.Permutation]'>
With iterative: Ainv of type <class 'cola.linalg.inv.GMResInv'>
Computed inverse in 101 iters with error 2.5e+01
Actual residual error: tensor(0.0027)


We can observe a few things here.
In the dense case, CoLA performs a PLU decomposition $B=P^{-1}LU$ and then computes the inverse as $U^{-1}L^{-1}P$ again using the implicit triangular solves, but this time combined with a permutation inverse.

In the iterative case, CoLA uses GMRES. Notably GMRES uses Arnoldi as a component of the algorithm, and the convergence criteria for Arnoldi depends on more than just residual errors that we care about for the linear solve. As a result of this more stringent convergence criteria, GMRES hits max_iters before stopping from the Arnoldi convergence tolerance, and the errors can be more easily controlled via changing max_iters.

In general CG has much more favorable properties over GMRES since with GMRES the memory requirement will grow with the number of iterations as will the compute, and hence if a matrix is PSD annotating it as such is highly preferable.

## Matrix Functions: $A^{1/2}$, $\exp(A)$, $\log(A)$, $f(A)$

Broadly we can consider many functions linear algebra operations $A^{1/2}$, $\exp(A)$, $\log(A)$
as instances of $f(A)$ where $f$ is assumed to have a convergent Taylor expansion within the bounds of the spectrum of $A$.

For 'method = dense', we compute these functions performing an eigendecomposition $A=P\Lambda P^{-1}$ and evaluate $f(A)=P f(\Lambda) P^{-1}$, and this runs in time $O(n^3)$.

For 'method = iterative', we evaluate matrix vector products $f(A)v$ using the Lanczos and Arnoldi processes starting with the vector $v$. Running in time $O(\tau \sqrt{\kappa}\log 1/\epsilon+m^3+mn)$ where $m$ is `max_iters`, this approach gives $\epsilon$ accurate evaluation of $f(A)v$. (With some extra effort the $m^3$ term can be reduced to $m^2$ but we have not yet implemented this optimization.)

For different $v$ the process will be run again, as Lanczos needs $v$ to construct a low error approximation.

### SelfAdjoint

Whether CoLA uses the more efficient Lanczos process or more costly version using Arnoldi depends on whether or not the matrix is `SelfAdjoint` (which gauruntees an orthogonal eigenbasis). `SelfAdjoint` is considered a superset of `PSD`.

In [8]:
print(A.annotations)
print(f"PSD: {A.isa(cola.PSD)}, SelfAdjoint: {A.isa(cola.SelfAdjoint)}")

{PSD}
PSD: True, SelfAdjoint: True


In [9]:
for method in ['dense', 'iterative']:
    S = cola.linalg.sqrt(A,tol=1e-4, max_iters=100, method=method)
    print(f"S with method={method} is of type {type(S)}")
    print("error in sqrt:", torch.linalg.norm(S@(S@x)-A@x)/torch.linalg.norm(A@x))

/home/ubu/venv/cola/lib/python3.10/site-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Callable deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


S with method=dense is of type <class 'cola.ops.operators.Product[cola.ops.operators.Dense, cola.ops.operators.Diagonal, cola.ops.operators.Dense]'>
error in sqrt: tensor(4.3223e-06)
S with method=iterative is of type <class 'cola.linalg.unary.LanczosUnary[cola.ops.operators.Sum[cola.ops.operators.Product[cola.ops.operators.Dense, cola.ops.operators.Dense], cola.ops.operators.Diagonal], function]'>
error in sqrt: tensor(1.2071e-05)


Likewise we can use `exp`, `log`, `pow`, and `apply_fn`

In [10]:
expA = cola.linalg.exp(-A)
logA = cola.linalg.log(A)
Apow10 = cola.linalg.pow(A,10)
resolvent = cola.linalg.apply_unary(lambda x: 1/(x-1), A)
[print(op[:2,:2].to_dense()) for op in [expA, logA, Apow10, resolvent]];

tensor([[0.3532, 0.0026],
        [0.0026, 0.3468]])
tensor([[ 0.0565, -0.0104],
        [-0.0104,  0.0613]])
tensor([[ 2.6618e+30, -5.8898e+29],
        [-5.8899e+29,  4.6795e+29]])
tensor([[37.2876,  4.1426],
        [ 4.1389, 18.6928]])


### Non SelfAdjoint
For non self adjoint matrices the situation is the same, but it will use Arnoldi iterations. However, some of the operations like `log` and `sqrt` require that eigenvalues are $>0$ and may return Nan, Inf, or complex values depending on the inputs.

In [11]:
ops = [cola.linalg.sqrt(B), cola.linalg.exp(-B), cola.linalg.pow(B,10)]
[print(op[:2,:2].to_dense()) for op in ops];

tensor([[0.5433-0.2177j, 0.1783+0.2979j],
        [0.0123+0.0198j, 0.9617+0.3228j]])
tensor([[ -966.8961-0.0044j, -3458.3843+0.0029j],
        [  502.8666+0.0023j,  1800.1865-0.0015j]])
tensor([[-4.5568e+19+7.3757e+13j, -2.5659e+19+1.9980e+14j],
        [ 1.6862e+19-2.5976e+13j,  9.4118e+18-7.5235e+13j]])


## Trace and Diagonal estimation

Unlike for Dense matrices, evaluating the trace or diagonal of a LinearOperators can be difficult (consider for example the diagonal of the Hessian of a neural network). Depending on the needs for the problem at hand we provide multiple different solutions:
- an exact $O(n\tau)$ compute and $O(\tau)$ memory evaluation which loops over the basis elements $A_{ii} = e_i^TAe_i$
- an stochastic Hutchinson estimator which is unbiased (always), but has runtime $O(1/\delta^2)$ where $\delta$ is the desired tolerance for the standard deviation of the estimate.

These can be chosen directly by specifying `method=exact`, `method=approx` or by letting the algorithm decide which will be faster based on the specified tolerance for the standard deviation `tol`. In general for methods which only need stochastic estimates (such as to be used in SGD) or only need one or two digits of precision, then the stochastic estimator will be faster.

We can use either of these two methods for
- computing the trace
- computing the diagonal
- computing off-diagonal diagonals

In [12]:
exact = cola.linalg.trace(A)
approx = cola.linalg.trace(A, method='approx', tol=3e-2)
print(f"exact trace: {exact}, stochastic trace: {approx}")

exact trace: 110965.4765625, stochastic trace: 110981.0


In [13]:
diag = cola.linalg.diag(A)
off_diag = cola.linalg.diag(A,k=1)
print(f"diagonal: {diag}")
print(f"off-diagonal: {off_diag}")

diagonal: tensor([  4.6068,   1.8986,  11.3133,  ..., 101.1196, 105.8028, 108.4426])
off-diagonal: tensor([-0.7107,  1.8426,  0.0810,  ...,  0.0902, -0.2768,  0.9027])


## log determinants

In the dense case, we compute log determinants from the Cholesky or LU decompositions depending on whether the matrix is PSD or not, and this runs in time $O(n^3)$.

For the iterative case we compute log determinants using the formula $\log\mathrm{det}(A) = \mathrm{Tr}(\log(A))$ combining together the $\log$ and $\mathrm{Tr}$ functions discussed above.

In the special case where only unbiased estimates (or a small number of significant digits) are required for the given use case, we can leverage [stochastic lanczos quadrature](https://www-users.cse.umn.edu/~saad/PDF/ys-2016-04.pdf) to get an improved convergence rate.

This choice is exposed to the user through the `method=iterative-stochastic` vs `method=iterative-exact` options in `logdet` and `slogdet`. If not chosen then they will be chosen automatically based on the selected `vtol` which chooses the "variance" tolerance specifying the standard deviation of the estimate.

For stochastic estimation, a key (`long` for pytorch, `PRNGKey` for jax) should be provided to explicitly to seed the randomness.

In [14]:
print("Tr(log(A))",cola.linalg.logdet(A, method='iterative', vtol=1e-1,tol=1e-4, max_iters=30, key=42))
print("SLQ:", cola.linalg.logdet(A, method='iterative', tol=1e-3,max_iters=10))
print("Dense:", cola.linalg.logdet(A, method='dense'))

Tr(log(A)) tensor(7319.7593)
SLQ: tensor(7324.9453)
Dense: tensor(7324.8359)


And we can do this for non PSD matrices too using $\log \mathrm{det}(A) = \frac{1}{2}\log \mathrm{det}(A^TA)$, however the phase (sign) is lost in this process.

In [15]:
print("iterative:", cola.linalg.slogdet(B, method='iterative', tol=1e-3,max_iters=10))
print("Dense:", cola.linalg.slogdet(B, method='dense'))

iterative: (ValueError('Unknown phase'), tensor(7329.1084))
Dense: (tensor(1.), tensor(7308.6660))


## 🚧 Eigendecomposition 🚧

Another popular linear algebra operation is to find an approximation for the eigenvalues or eigenvectors of given linear operator $A$. That is, find $V$ and $\Lambda$ such that $AV = V \Lambda$.

In [16]:
eig_vals, eig_vecs = cola.eig(A, method='dense')

Nevertheless, when $A$ is quite large we cannot afford the time or memory for a full decomposition. We could thus use an iterative eigenvalue algorithm such as power iteration (if we only want the maximum eigenvalue), Lanczos (if we have a symmetric operator) or Arnoldi (which works in general). Let's see how to run all of those options.
So to compute the maximum eigenvalue we could run

In [17]:
eig_max = cola.eigmax(A, tol=1e-4)
print(f"eigmax: {eig_max}")
print(f"eig_vals[0]: {eig_vals[-1]}")

eigmax: 2164.6376953125
eig_vals[0]: 2166.60107421875


Indeed we recover the largest eigenvalue. The algorithm that `CoLA` ran in this case was the power method which runs in time $O(\tau\frac{1}{\Delta} \log 1/\epsilon)$ to compute the top eigenvalue. But we can achieve an accelerated convergence rate $O(\tau\frac{1}{\sqrt{\Delta}} \log 1/\epsilon)$ if we use Lanczos and Arnoldi, however this comes at an additional memory cost of $O(nm+m^2)$ where $m$ is `max_iters`.

In [18]:
import numpy as np
e0, v0 = cola.eig(A, num=None, which="SM", method="iterative", tol=1e-4, max_iters=15)
print(f"{e0[-1]}")

2166.601806640625


## 🚧 SVD (under development) 🚧

## 🚧 Pseudoinverse (under development) 🚧